### **Python - Тестирование**

На лекции поговорим о том, зачем нужны тесты и как их писать. Рассмотрим способы улучшения качества кода и избавления его от багов с помощью различных инструментов: от линтеров до E2E-тестов.
Вы узнаете или вспомните, как использовать flake8, mypy, pytest, fixture, coverage, mock, factory boy, faker и многое другое.

In [1]:
#vibo: первоисточник https://www.youtube.com/watch?v=957lkNw-ThE

#### **=== Static Analisis ===** 
Статические анализаторы (линтеры). Код не запускается. Проверяется синтаксис, неисползуемые переменные, импорты.

##### **-- pep8 aka pycodestyle**

In [2]:
%pip --version

pip 22.2.2 from /home/vibo/vs_code/venv-vsc/lib/python3.9/site-packages/pip (python 3.9)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pep8

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade pep8

Note: you may need to restart the kernel to use updated packages.


In [ ]:

'''
#vibo: пример кода, сохраняем в main0.py

import sys

def f1(first_long_parameter, second_long_parameter, third_very_long_long_parameter):
    b = a + 1
'''

In [3]:
!python -m pep8 main0.py

/home/vibo/vs_code/venv-vsc/lib/python3.9/site-packages/pep8.py:110: FutureWarning: Possible nested set at position 1
  EXTRANEOUS_WHITESPACE_REGEX = re.compile(r'[[({] | []}),;:]')
/home/vibo/vs_code/venv-vsc/lib/python3.9/site-packages/pep8.py:2123: UserWarning: 

pep8 has been renamed to pycodestyle (GitHub issue #466)
Use of the pep8 tool will be removed in a future release.
Please install and use `pycodestyle` instead.

$ pip install pycodestyle
$ pycodestyle ...

  warnings.warn(
main0.py:3:1: E302 expected 2 blank lines, found 1
main0.py:3:80: E501 line too long (84 > 79 characters)


In [6]:
%pip install pycodestyle

Note: you may need to restart the kernel to use updated packages.


##### **-- flake8**

In [7]:
%pip install flake8

Note: you may need to restart the kernel to use updated packages.


In [5]:
!python -m flake8 main0.py

main0.py:1:1: F401 'sys' imported but unused
main0.py:3:1: E302 expected 2 blank lines, found 1
main0.py:3:80: E501 line too long (84 > 79 characters)
main0.py:4:5: F841 local variable 'b' is assigned to but never used
main0.py:4:9: F821 undefined name 'a'


+flake8 имеет ряд полезных плагионов

##### **-- pylint**

In [8]:
%pip install pylint

Note: you may need to restart the kernel to use updated packages.


In [9]:
!python -m pylint main0.py

************* Module main0
main0.py:1:0: C0114: Missing module docstring (missing-module-docstring)
main0.py:3:0: C0116: Missing function or method docstring (missing-function-docstring)
main0.py:3:0: C0103: Function name "f1" doesn't conform to snake_case naming style (invalid-name)
main0.py:4:4: C0103: Variable name "b" doesn't conform to snake_case naming style (invalid-name)
main0.py:4:8: E0602: Undefined variable 'a' (undefined-variable)
main0.py:3:7: W0613: Unused argument 'first_long_parameter' (unused-argument)
main0.py:3:29: W0613: Unused argument 'second_long_parameter' (unused-argument)
main0.py:3:52: W0613: Unused argument 'third_very_long_long_parameter' (unused-argument)
main0.py:4:4: W0612: Unused variable 'b' (unused-variable)
main0.py:1:0: W0611: Unused import sys (unused-import)

-----------------------------------
Your code has been rated at 0.00/10



##### **-- autopep8** (форматер)

In [10]:
%pip install autopep8

Note: you may need to restart the kernel to use updated packages.


##### **-- black** (форматер)

In [11]:
%pip install black

Note: you may need to restart the kernel to use updated packages.


In [12]:
!black main0.py

reformatted main0.py

All done! ✨ 🍰 ✨
1 file reformatted.


In [14]:
!black -S -l 79 main0.py

reformatted main0.py

All done! ✨ 🍰 ✨
1 file reformatted.


С black нужно быть аккуратным, применяя комментарий # fmt:off ... # fmt:on можно отключить автоформатирование. Количество настроек balk - минимально (версия питона, длинная строки, какие файлы нужно/ненужно форматировать).  

In [15]:
!black -S -l 79 main0_fmt_black.py

All done! ✨ 🍰 ✨
1 file left unchanged.


#### **Аннотации типов (тайпинги) + mypy**

In [16]:
#vibo: код без тайпингов
def indent_right(s, width):
    return " " * (max(0, width - len(s))) + s

class Book:
    def __init__(self, title, author, cost):
        self.title = title
        self.author = author
        self.cost = cost

b = Book(title='Fahrenheit 451', author='Bradbury', cost='3,14')

In [17]:
#vibo: код с тайпингами (с версии Python 3.5)
def indent_right(s: str, width: int) -> str:
    return " " * (max(0, width - len(s))) + s

class Book:
    title: str
    author: str

    def __init__(self, title: str, author: str, cost: int) -> None:
        self.title = title
        self.author = author
        self.cost = cost

b = Book(title='Fahrenheit 451', author='Bradbury', cost='3,14')

In [18]:
%pip install mypy

Note: you may need to restart the kernel to use updated packages.


In [19]:
!mypy main1_typing.py

main1_typing.py:13: error: Argument "cost" to "Book" has incompatible type "str"; expected "int"
Found 1 error in 1 file (checked 1 source file)


mypy находит по указанным тайпингам ошибки не соответствия типов данных, которые бы не заметил python.

#### **=== Unit ===**
Юнит-тесты предусматривают запуск в Python частей кода (модули/классы/функции).

##### **-- assert**

In [21]:
#vibo: нормальная функция сртировки списка
def _sort(lst):
    return sorted(lst)

#vibo: "сломанная" функция сортировки списка
def _broken_sort(lst):
    return sorted(lst[1:])

#vibo: функция тестирования сортировки
def test_sort():
    #vibo: задаем список
    lst = [1, 7, 2, 9, 3, 8, 4]
    #vibo: используем оператор assert, провекрка и комментарий
    assert _sort(lst) == [1, 2, 3, 4, 7, 8, 9], "_sort fails"
    assert _broken_sort(lst) == [1, 2, 3, 4, 7, 8, 9], "_broken_sort fails"

if __name__ == '__main__':
    test_sort() 

AssertionError: _broken_sort fails

Так (один оператор assert) не очень удобно, т.к. "_broken_sort fails" не пойми о чем говорит, тест упал и все, дальше проверка не идет

##### **-- pytest + assert**

есть doctest, unitest и pytest;
pytest более современный, поддерживает тесты unitest, куча полезных плагинов, например, тот же flake8, coverage (покрытие тестами кода)

In [22]:
%pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [23]:
!pytest main2_assert.py

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 1 item                                                               

main2_assert.py F                                                        [100%]

=================================== FAILURES ===================================
__________________________________ test_sort ___________________________________

    def test_sort():
        lst = [1, 7, 2, 9, 3, 8, 4]
        assert _sort(lst) == [1, 2, 3, 4, 7, 8, 9], "_sort fails"
>       assert _broken_sort(lst) == [1, 2, 3, 4, 7, 8, 9], "_broken_sort fails"
E       AssertionError: _broken_sort fails
E       assert [2, 3, 4, 7, 8, 9] == [1, 2, 3, 4, 7, 8, ...]
E         At index 0 diff: 2 != 1
E         Right contains one more item: 9
E         Use -v to get m

pytest реагирует на все функции, начинающиеся с test_, конкретно указывает на ошибку  assert [2, 3, 4, 7, 8, 9] == [1, 2, 3, 4, 7, 8, ...]; еще более подробная информация с -v

In [25]:
!pytest main2_assert.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 1 item                                                               

main2_assert.py::test_sort FAILED                                        [100%]

=================================== FAILURES ===================================
______________________

##### **-- pytest: raises, xfail, skipif** 
(набор функций декораторов, которые помогают тесты писать)

In [43]:
import pytest
import sys


#vibo: декоратор raises (проверка исключения) - менеджер контекста
def test_raises():
    with pytest.raises(IndexError):
        kth_stat(1, 0)

#vibo: декоратор xfail (позволяет пометить тест как сломаный)
@pytest.mark.xfail()
def test_raises():
    kth_stat([1, 2, 3], 100)

#vibo: декоратор skipif (позволяет пропустить тест, darwin - MacOS)
@pytest.mark.skipif(sys.platform == 'linux', reason='don\'t know why, but may fail on linux')
def test_not_to_run_on_linux(filld_file):
    assert kth_stat(json.load(filled_file), 500) == 499

In [44]:
!pytest main3_pytest1.py --collect-only

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 2 items                                                              

<Module main3_pytest1.py>
  <Function test_raises>
  <Function test_not_to_run_on_linux>

========================== 2 tests collected in 0.01s ==========================


In [46]:
!pytest main3_pytest1.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 2 items                                                              

main3_pytest1.py::test_raises XFAIL                                      [ 50%]
main3_pytest1.py::test_not_to_run_on_linux SKIPPED (don't know why, ...) [100%]

=======================

pytest: полезные опции

--collect-only - вывод списка найденных тестов
-k - фильтрация по имени теста
-s - включает вывод stdout & stderr тестов (по умолчанию выводятся только для упавших тестов)
-v - повышает детализацию процесса запуска тестов
--lf, --lasted-failed - перезапускает тесты, упавшие при последнем запуске
--sw, --stepwise - выходит при падении и при последующих запусках продолжает с последнего упавшего теста

##### **-- Fixtures (фикстуры)** 
(когда нужно подготовить данные, фикстура - функция, помеченная специальным декоратором fixture, которую мы можем передать в функцию в качестве параметра)

In [17]:
import pytest
import tempfile
import json
import random


def kth_stat(lst, n):
    #vibo: сортируем список, берем значение по n-ому индексу
    return sorted(lst)[n]

#vibo: готовим данные, декоратор fixture
@pytest.fixture
def filled_file():
    #vibo: открываем временный файл
    with tempfile.TemporaryFile(mode='w+') as f:
        #vibo: создаем список
        li = list(range(10000))
        #vibo: перемешиваем список
        random.shuffle(li)
        #vibo: записываем во временный файл
        json.dump(li, f)
        #vibo: ставим указатель в начало
        f.seek(0)
        #vibo: параметр f передается в качестве парамтра в тестируемую функцию
        yield f

#vibo: передаем подготовленные данные (фикстуру filled_file) в тестируемую функцию
#vibo: если мы подключили фикстуру к тесту, то сначала выолнитеся фикстура
def test_on_large_seq_from_file(filled_file):
    #vibo: ломаем
    assert kth_stat(json.load(filled_file), 300) == 299
    #vibo: чиним
    #assert kth_stat(json.load(filled_file), 300) == 300

In [52]:
!pytest main3_pytest2.py

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 1 item                                                               

main3_pytest2.py F                                                       [100%]

=================================== FAILURES ===================================
_________________________ test_on_large_seq_from_file __________________________

filled_file = <_io.TextIOWrapper name=11 mode='w+' encoding='UTF-8'>

    def test_on_large_seq_from_file(filled_file):
>       assert kth_stat(json.load(filled_file), 300) == 299
E       AssertionError: assert 300 == 299
E        +  where 300 = kth_stat([7932, 4913, 2572, 6658, 8403, 5720, ...], 300)
E        +    where [7932, 4913, 2572, 6658, 8403, 5720, ...] = <function load at 0x7f17117a6160>(<_io.Te

In [53]:
!pytest main3_pytest2.py

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 1 item                                                               

main3_pytest2.py .                                                       [100%]

============================== 1 passed in 0.07s ===============================


##### **--- Фикстуры разного уровня**
- scope='module' -  фикстура вызывется один раз на модуль, т.е. если в модуле несколько тестов она будет вызываться один раз;
- () - фикстура без параметров вызывается каждый раз, когда она используется;
- autouse=True - вызывается даже тогда, когда в явном виде ее не подключаем

In [55]:
#vibo: фикстура вызывется один раз на модуль, т.е. если в модуле несколько тестов она будет вызываться один раз
@pytest.fixture(scope='module')
def call_me_once_use_when_needed():
    print('\ncall me once use when needed')

#vibo: фикстура без параметров вызывается каждый раз, когда она используется
@pytest.fixture()
def call_me_every_time():
    print('call me every time')

#vibo: вызывается даже тогда, когда в явном виде ее не подключаем
@pytest.fixture(autouse=True)
def call_me_everywhere():
    print('YOU\'LL CALL ME EVEN IF YOU DON\'T WANNA TO')

def test_one(call_me_once_use_when_needed, call_me_every_time):
    print('test one')

def test_two(call_me_once_use_when_needed, call_me_every_time):
    print('test two')

In [54]:
!pytest main3_pytest3.py -s

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 2 items                                                              

main3_pytest3.py 
call me once use when needed
YOU'LL CALL ME EVEN IF YOU DON'T WANNA TO
call me every time
test one
.YOU'LL CALL ME EVEN IF YOU DON'T WANNA TO
call me every time
test two
.

============================== 2 passed in 0.05s ===============================


##### **--- Фикстуры можно 'наследовать'** 
(передавать внутрь другой фикстуры)

In [61]:
@pytest.fixture
def init_db():
    print('\ninit_db')

@pytest.fixture
def run_migrations(init_db):
    print('run_migrations')

@pytest.fixture
def superuser(run_migrations):
    print('create superuser')

def test_one(superuser):
    print('test one')

def test_two(run_migrations):
    print('test two')

In [62]:
!pytest main3_pytest4.py -s

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0
collected 2 items                                                              

main3_pytest4.py 
init_db
run_migrations
create superuser
test one
.
init_db
run_migrations
test two
.

============================== 2 passed in 0.04s ===============================


##### **-- conftest.py**
(фикстуры из этого файла становятся доступны из всех файлов; если не понятно откуда взялись фикстуры - смотри файл conftest.py)

##### **-- pytest: плагины**

- flake8 (проверка синтаксиса);
- coverage (покрытие кода тестами, делает красивый html-отчет);
- pytest-sugar (более красивый вывод);
- django (декоратор, поднимающий окружение django, для flask тоже есть свой плагин);
- xdist (для параллельного запуска на нескольких cpu; главное, чтобы тесты друг другу не мешали);
- timeout (таймаут для теста).

pytest можно настраивать, в файле .ini

In [2]:
#vibo: coverage
!pip install pytest-cov

In [ ]:
!ls

In [ ]:
#vibo: выводом на экран
!pytest --cov=src

In [ ]:
#vibo: с выводом в html формат
!pytest --cov=src --cov-report=html

##### **-- Параметризация (декоратор parametrize)**

In [8]:
#vibo: если хотим не просто запустить тест, а с разными параметрами
import pytest


def kth_stat(lst, n):
    return sorted(lst)[n]

#vibo: несколько параметров для тестирования
@pytest.mark.parametrize(
    ('values', 'stat_order', 'expected'), [
        ([1], 0, 1),
        ([1, 1, 1, 1, 1], 4, 1),
        (range(100), 4, 3)
    ]
)
def test_on_range(values, stat_order, expected):
    assert kth_stat(values, stat_order) == expected

In [10]:
!pytest main3_pytest5.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0', 'cov': '3.0.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 3 items                                                              

main3_pytest5.py::test_on_range[values0-0-1] PASSED                      [ 33%]
main3_pytest5.py::test_on_range[values1-4-1] PASSED                      [ 66

##### **-- hypothesis (параметризация для "ленивых")**

In [11]:
%pip install hypothesis

Note: you may need to restart the kernel to use updated packages.


In [19]:
from hypothesis import given
from hypothesis.strategies import lists, integers

def broken_sort(it):
    #vibo: вставляем баг (не сортировать при длине списка 5)
    if len(it) == 5:
        return it
    return sorted(it)

#vibo: декоратор given генерирует набор
@given(lists(integers()))
def test_sort(it):
    assert broken_sort(it) == sorted(it)

In [20]:
!pytest main4_hypothesis.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0', 'cov': '3.0.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 1 item                                                               

main4_hypothesis.py::test_sort FAILED                                    [100%]

=================================== FAILURES ===============================

#### **=== Integration ===**
Интеграционные тесты. Проверяется работа/взаимодействие нескольких систем. Тестирование группы взаимодействующих модулей/программ.

##### **-- unittest.mock (подменяет внешние объекты или функции)**

для вызова mok используем библиотеку unittest (аналог mok есть и у библиотеки pytest, которую смотрели выше)

In [40]:
from unittest.mock import Mock

#vibo: создаем объект Mock()
m = Mock()

In [44]:
#vibo: вызываем объект Mock(), никакой ошибки нет
m()

<Mock name='mock()' id='140163388131696'>

In [46]:
#vibo: можем вызвать метод у этого объекта
m.f()

<Mock name='mock.f()' id='140163388132416'>

In [47]:
#vibo: можем вызвать параметр у этого объекта
m.is_alive

<Mock name='mock.is_alive' id='140163385804352'>

Итого можем подменить внешний сервис и имитировать его поведение. Но и можем проверить, что наш код туда попытался сходить.

In [48]:
#vibo: с помощью метода call_count проверяем сколько раз вызывался объект m
m.call_count

4

In [49]:
#vibo: с помощью метода call_count проверяем сколько раз вызывался метод f объекта m 
m.f.call_count

2

In [50]:
#vibo: есть класс, который ходит во внешний сервис; этот класс нужно протестировать
from unittest.mock import Mock
class AliveChecker:
    #vibo: подаем в конструктор http-сессию и target - dns-сессию куда нужно сходить
    def __init__(self, http_session, target):
        self.http_session = http_session
        self.target = target
    
    #vibo: метод класса 
    def do_check(self):
        try:
            resp = self.http_session.get(
                f'https://{self.target}/ping')
        except Exception:
            return False
        else:
            #vibo: если все хорошо
            return resp == 200

Как мы можем протестировать класс AliveChecker, который ходит во внешний сервис. Решаем спомощью mock.

In [61]:
#vibo: тестируем с помощью mock (ПОЗИТИВНЫЙ СЦЕНАРИЙ)
def test_with_mock():
    #vibo: создаем mock на метод get, говорим, что возвращать должен 200
    get_mock = Mock(return_value=200)
    #vibo: также с помощью mock создаем псевдо-клиента
    pseudo_client = Mock()
    #vibo: говорим, что метод get у псевдо-клиента это созданный нами get_mock
    pseudo_client.get = get_mock
    #vibo: создаем подменный alive_checker, подаем в оригинальный AliveChecker
    alive_checker = AliveChecker(pseudo_client, 'test.com')
    
    #vibo: вызываем метод do_check(), чтобы проверить работоспособность
    assert alive_checker.do_check()
    #vibo: вызываем assert_called_once_with, чтобы понять что мы его точно вызывали
    pseudo_client.get.assert_called_once_with('https://test.com/ping')

In [62]:
#vibo: тестируем с помощью mock (НЕГАТИВНЫЙ СЦЕНАРИЙ)
def test_with_raising_mock():
    get_mock = Mock(side_effect=Exception('EEEEE'))
    pseudo_client = Mock()
    pseudo_client.get = get_mock
    alive_checker = AliveChecker(pseudo_client, 'test.com')
    assert not alive_checker.do_check()
    pseudo_client.get.assert_called_once_with('https://test.com/ping')

In [70]:
!pytest main5_unittest_mock1.py -v -s

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0', 'cov': '3.0.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 1 item                                                               

main5_unittest_mock1.py::test_with_raising_mock PASSED

============================== 1 passed in 0.20s ===============================


##### **-- mock: патчим методы**

In [76]:
def test_classroom_post_save(self):
    #vibo: прописываем весь путь до метода и делаем его объектом mock
    with patch('calendars.recievers.create_layer.delay') as mock:
        ClassroomFactory()
        #vibo: проверяем (замокали объект); метод реально вызыватьсz не будет, а будет подставляться mock
        #vibo: вызываем фабрику и проверяем, что метод mock не вызывался
        mock.assert_not_called()

        #vibo: создаем календарь, и что он вызывался
        classroom = ClassroomFactory(calendar_enabled=True)
        mock.assert_called_once_with(course_id=classroom.course_id)

        #vibo: сохранение тоже можно проверить
        classroom.save()
        mock.assert_called_once_with(course_id=classroom.course_id)

In [85]:
!pytest main5_unittest_mock2.py -s

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 1 item                                                               

main5_unittest_mock2.py F

=================================== FAILURES ===================================
___________________________ test_classroom_post_save ___________________________

    def test_classroom_post_save():
>       with patch('calendars.recievers.create_layer.delay') as mock:

main5_unittest_mock2.py:4: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
/usr/lib/python3.9/unittest/mock.py:1388: in __enter__
    self.target = self.getter()
/usr/lib/python3.9/unittest/mock.py:1563: in <lambda>
    getter = lambda: _importer(target)
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

##### **-- mock: патчим библиотеки**

In [86]:
import math
from unittest.mock import patch

def test_patch_sin():
    with patch('math.sin', return_value=2) as m:
        assert math.sin(0) == 2
        assert math.sin(1) == 2
        assert m.call_count == 2

In [91]:
!pytest main5_unittest_mock3.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0', 'cov': '3.0.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 1 item                                                               

main5_unittest_mock3.py::test_patch_sin PASSED                           [100%]

============================== 1 passed in 0.25s ===========================

##### **-- freezegun (мокаем текущее время)**

In [71]:
%pip install freezegun

Note: you may need to restart the kernel to use updated packages.


In [97]:
#vibo: если нужно замокать текущее время
from freezegun import freeze_time
import datetime

#Freeze time for a pytest test:

@freeze_time("2022-01-14")
def test():
    now = datetime.datetime.now()
    assert now == datetime.datetime(2022, 1, 14)

In [96]:
!pytest main6_unittest_time.py -v

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.1.2, pluggy-1.0.0 -- /home/vibo/vs_code/venv-vsc/bin/python
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase('/home/vibo/vs_code/.hypothesis/examples')
metadata: {'Python': '3.9.9', 'Platform': 'Linux-5.15.59-1-MANJARO-x86_64-with-glibc2.33', 'Packages': {'pytest': '7.1.2', 'py': '1.11.0', 'pluggy': '1.0.0'}, 'Plugins': {'hypothesis': '6.54.1', 'Faker': '13.15.1', 'metadata': '2.0.2', 'allure-pytest': '2.9.45', 'json-report': '1.5.0', 'cov': '3.0.0'}}
rootdir: /home/vibo/vs_code
plugins: hypothesis-6.54.1, Faker-13.15.1, metadata-2.0.2, allure-pytest-2.9.45, json-report-1.5.0, cov-3.0.0
collected 1 item                                                               

main6_unittest_time.py::test PASSED                                      [100%]

============================== 1 passed in 0.26s ===========================

##### **-- vsr (mock http-запросов)**

In [100]:
%pip install django.test

  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for django.test, since package 'wheel' is not installed.
  Running setup.py install for django.test ... done
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from django.test import override_settings

from ..exceptions import GetScoreException
from ..services import get_student_score

@vcr.use_cassette("simple_score/200.yaml")
def test_get_student_score():
    assert get_student_score(1) == 21

При первом запросе ответ сохраняется в yaml-файле.
При повторных запросах ответ берется из yaml-файла.
Yaml-файл можно закоммитить в репозиторий.

##### **-- Полезные модули**

Сегодя рассмотрели:
- flake8;
- black;
- mypy;
- pytest;
- mock.

Могут пригодиться:
- factory_boy (расширение для фреймворков, есть для django, allchimi);
- faker (генерирует фейковые данные).

#### **=== E2E ====**
End to End тесты, проверяют реальные сценарии взаимодействия с пользователем.

#### **Какие тесты еще бывают:**

- Smoke (тесты для минимальной функциональности, работает или нет);
- Regression (тесты, позволяющие проверить при вводе новой функциональности не сломалась ли старая);
- Compatibilit (тесты совместимости);
- Installation (тесты установки);
- Acceptance (приемочные тесты);
- Alpha/Beta (на разных категориях пользователей);
- Performance (нагрузочное тестирование);
- Stress (нагрузочное тестирование);
- ...


#### **Полезные мысли**:

- не мокайте весь сервис -> растет процент ложноотрицательных тестов;
- не пишите тесты, которые завязаны на сеть или рандом -> растет процент ложноположительных тестов;
- не старайтесь писать тесты на 100%-ное покрытие. 20% усилий дают 80% успеха :);
- если в компании не было тестов... не нужно бросаться писать тесты для всего, начните добавлять по мере написания новой функциональности;
- тест проверяет либо отдельный метод (unit, интеграционный), либо сценарий (e2e);
- проверяйте количество обращений к базе/сервису. Чтобы отлавливать лишние запросы;
- если в функции много if - с применением параметризации тестов нужно зайти в каждый;
- тестировать код на максимально реалистичном сценарии;
- разбивать тесты по группам (отдельный файл на каждый тест);
- старайтесь писать простые тесты!
